In [13]:
import os
from dotenv import load_dotenv, find_dotenv
from mp_api.client import MPRester
import torch
import torch.nn as nn
import torch.optim as optim
import torch_geometric
import e3nn
from e3nn import o3
import e3nn.util.datatypes
import numpy as np
import pickle
from mendeleev import element
import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import io
import random
import math
import sys
import time, os
import datetime
from pathlib import Path

# Load environment variables
load_dotenv()
load_dotenv(Path("/Users/abiralshakya/Documents/Research/Topological_Insulators_OnGithub/generative_nmti/Integrated_Magnetic_Topological/matprojectapi.env"))
api_key = os.getenv("MP_API_KEY")

# Initialize variables for storing data
order_list_mp = []
structures_list_mp = []
formula_list_mp = []
sites_list = []
id_list_mp = []
y_values_mp = []
order_encode = {"NM": 0, "AFM": 1, "FM": 2, "FiM": 2}

# Initialize MPRester client
m = MPRester(api_key=api_key)

# Define element batches for querying
element_batches = [
    ["Ga", "Tm", "Y", "Dy", "Nb", "Pu"],
    ["Th", "Er", "U", "Cr", "Sc", "Pr"],
    ["Re", "Ni", "Np", "Nd", "Yb", "Ce"],
    ["Ti", "Mo", "Cu", "Fe", "Sm", "Gd"],
    ["V", "Co", "Eu", "Ho", "Mn", "Os"],
    ["Tb", "Ir", "Pt", "Rh", "Ru"]
]

# Storage for results
all_materials = []

# Fetch data in batches with proper approach
for batch in element_batches:
    try:
        print(f"Querying elements: {', '.join(batch)}")
        
        # Process each element separately
        for element in batch:
            print(f"  - Querying for materials with {element}")
            
            # Using the standard query format
            docs = m.materials.summary.search(
                chemsys=element,  # Use chemsys to search for the element
                fields=[
                    "material_id", 
                    "formula_pretty", 
                    "structure", 
                    "nsites", 
                    "is_magnetic", 
                    "ordering",
                    "total_magnetization"
                ]
            )
            
            # Add results to collection
            element_docs = list(docs)
            print(f"    Found {len(element_docs)} materials")
            all_materials.extend(element_docs)
            
            # Pause briefly to avoid rate limiting
            time.sleep(0.5)
    
    except Exception as e:
        print(f"Error processing batch {batch}: {e}")

print(f"Total materials retrieved: {len(all_materials)}")

# Process retrieved data
for doc in all_materials:
    try:
        material_id = doc.material_id
        formula = doc.formula_pretty
        structure = doc.structure
        nsites = doc.nsites
        
        # Extract magnetic ordering - directly from the ordering field
        magnetic_ordering = None
        if hasattr(doc, 'ordering') and doc.ordering:
            magnetic_ordering = doc.ordering
        
        # Store data in lists
        id_list_mp.append(material_id)
        formula_list_mp.append(formula)
        structures_list_mp.append(structure)
        sites_list.append(nsites)
        
        # Handle magnetic ordering data
        if magnetic_ordering and magnetic_ordering in order_encode:
            order_list_mp.append(magnetic_ordering)
            y_values_mp.append(order_encode[magnetic_ordering])
        else:
            # If structure not magnetic or ordering unknown
            if hasattr(doc, 'is_magnetic') and not doc.is_magnetic:
                order_list_mp.append("NM")  # Non-magnetic
                y_values_mp.append(order_encode["NM"])
            else:
                order_list_mp.append("Unknown")
                y_values_mp.append(-1)  # Mark as unknown
            
    except Exception as e:
        print(f"Error processing material {doc.material_id if hasattr(doc, 'material_id') else 'unknown'}: {e}")

# Remove duplicate materials (if any)
unique_indices = []
seen_ids = set()
for i, material_id in enumerate(id_list_mp):
    if material_id not in seen_ids:
        seen_ids.add(material_id)
        unique_indices.append(i)

if len(unique_indices) < len(id_list_mp):
    print(f"Removing {len(id_list_mp) - len(unique_indices)} duplicate materials")
    id_list_mp = [id_list_mp[i] for i in unique_indices]
    formula_list_mp = [formula_list_mp[i] for i in unique_indices]
    structures_list_mp = [structures_list_mp[i] for i in unique_indices]
    sites_list = [sites_list[i] for i in unique_indices]
    order_list_mp = [order_list_mp[i] for i in unique_indices]
    y_values_mp = [y_values_mp[i] for i in unique_indices]

print(f"Processed {len(id_list_mp)} unique materials with structures")
print(f"Materials with known magnetic ordering: {sum(1 for y in y_values_mp if y >= 0)}")

# Optional: Create a summary of magnetic orders
mag_order_counts = {}
for order in order_list_mp:
    if order in mag_order_counts:
        mag_order_counts[order] += 1
    else:
        mag_order_counts[order] = 1

print("Magnetic ordering distribution:")
for order, count in mag_order_counts.items():
    print(f"  {order}: {count}")

Querying elements: Ga, Tm, Y, Dy, Nb, Pu
  - Querying for materials with Ga


Retrieving SummaryDoc documents: 100%|██████████| 8/8 [00:00<00:00, 216480.21it/s]


    Found 8 materials
  - Querying for materials with Tm


Retrieving SummaryDoc documents: 100%|██████████| 5/5 [00:00<00:00, 161319.38it/s]


    Found 5 materials
  - Querying for materials with Y


...

    Found 3 materials
  - Querying for materials with Rh


Retrieving SummaryDoc documents: 100%|██████████| 5/5 [00:00<00:00, 81920.00it/s]


    Found 5 materials
  - Querying for materials with Ru


Retrieving SummaryDoc documents: 100%|██████████| 2/2 [00:00<00:00, 38479.85it/s]


    Found 2 materials
Total materials retrieved: 211
Processed 211 unique materials with structures
Materials with known magnetic ordering: 211
Magnetic ordering distribution:
  NM: 137
  FM: 59
  FiM: 11
  AFM: 4


In [14]:
import torch
from datetime import datetime

In [15]:
time_stamp = datetime.today().strftime('%Y-%m-%d_%H-%M')

In [16]:
struct_dict = {
    "structures": structures_list_mp
}
print(struct_dict)

{'structures': [Structure Summary
Lattice
    abc : 2.7784220848958743 3.28160879 4.210354752082499
 angles : 90.0 108.60716858150202 90.0
 volume : 36.382091139482284
      A : np.float64(2.62238406) np.float64(0.0) np.float64(-0.91800388)
      B : np.float64(0.0) np.float64(3.28160879) np.float64(0.0)
      C : np.float64(0.05042213) np.float64(0.0) np.float64(4.21005282)
    pbc : True True True
PeriodicSite: Ga (2.306, 0.8204, 2.278) [0.8654, 0.25, 0.7298]
PeriodicSite: Ga (0.3666, 2.461, 1.014) [0.1346, 0.75, 0.2702], Structure Summary
Lattice
    abc : 3.0362201016259225 3.0362201016259225 7.9166609999999995
 angles : 90.0 90.0 99.98230381819225
 volume : 71.87595697275971
      A : np.float64(1.95200382) np.float64(-2.32557812) np.float64(0.0)
      B : np.float64(1.95200382) np.float64(2.32557812) np.float64(0.0)
      C : np.float64(0.0) np.float64(0.0) np.float64(7.9166609999999995)
    pbc : True True True
PeriodicSite: Ga (1.952, 1.302, 0.9952) [0.2201, 0.7799, 0.1257]
Per

In [17]:
torch.save(struct_dict, f'mp_structures_{time_stamp}.pt')

In [19]:
from pymatgen.core.structure import Structure
import torch.serialization

# Add Structure class to the safe list
torch.serialization.add_safe_globals([Structure])

# Then load your file
structures_loaded = torch.load(f'mp_structures_{time_stamp}.pt', weights_only=False)

In [23]:
f = torch.load('/Users/abiralshakya/Documents/Research/Topological_Insulators_OnGithub/generative_nmti/Integrated_Magnetic_Topological/magnetic_order/preload_data/mp_structures_2025-04-07_11-39.pt', weights_only= False)

In [24]:
print(f)

{'structures': [Structure Summary
Lattice
    abc : 2.7784220848958743 3.28160879 4.210354752082499
 angles : 90.0 108.60716858150202 90.0
 volume : 36.382091139482284
      A : np.float64(2.62238406) np.float64(0.0) np.float64(-0.91800388)
      B : np.float64(0.0) np.float64(3.28160879) np.float64(0.0)
      C : np.float64(0.05042213) np.float64(0.0) np.float64(4.21005282)
    pbc : True True True
PeriodicSite: Ga (2.306, 0.8204, 2.278) [0.8654, 0.25, 0.7298]
PeriodicSite: Ga (0.3666, 2.461, 1.014) [0.1346, 0.75, 0.2702], Structure Summary
Lattice
    abc : 3.0362201016259225 3.0362201016259225 7.9166609999999995
 angles : 90.0 90.0 99.98230381819225
 volume : 71.87595697275971
      A : np.float64(1.95200382) np.float64(-2.32557812) np.float64(0.0)
      B : np.float64(1.95200382) np.float64(2.32557812) np.float64(0.0)
      C : np.float64(0.0) np.float64(0.0) np.float64(7.9166609999999995)
    pbc : True True True
PeriodicSite: Ga (1.952, 1.302, 0.9952) [0.2201, 0.7799, 0.1257]
Per